In [1]:
%load_ext autoreload
%autoreload

import json
import bogglesolver as boggle
import numpy as np

CHARSET = list('abcdefghijklmnopqrstuvwxyz')

with open('words_dictionary.json') as data_file:    
    data = json.load(data_file)
#data = []
#with open('sowpods.txt') as f:
#    for line in f:
#        data.append(line)
    
all_words = []
for d in data:
    if all(list(map(lambda x: x in CHARSET, list(d)))):
        all_words.append(d)


In [2]:
print("word count: "+ str(len(all_words)))

letter_count=0
for t in all_words:
    letter_count=letter_count+len(t)    
print("avg word length: "+str(letter_count/len(all_words)))

vowels = list('aeiou')
vow_count = 0
for w in all_words:
    for l in w:
        if l in vowels:
            vow_count=vow_count+1
VOWEL_P = vow_count/letter_count
print("vowel fraction: "+str(VOWEL_P))

LETTER_DISTR = {}
for l in list('abcdefghijklmnopqrstuvwxyz'):
    LETTER_DISTR[l] = 0
for t in all_words:
    for l in t:
        if l in list('abcdefghijklmnopqrstuvwxyz'):
            LETTER_DISTR[l] = LETTER_DISTR[l]+1
for l in LETTER_DISTR:
    LETTER_DISTR[l]=LETTER_DISTR[l]/letter_count
    
vv = 0
vc = 0
cc = 0
cv = 0
def is_vowel(x):
    return x in list('aeiou')
def is_consonant(x):
    return x in list('bcdfghjklmnpqrstvwxyz')

for w in all_words:
    for i in range(0, len(w)-1):
        a = w[i]
        b = w[i+1]
        if is_vowel(a):
            if is_vowel(b):
                vv = vv+1
                continue
            if is_consonant(b):
                vc = vc+1
                continue
        if is_consonant(a):
            if is_vowel(b):
                cv = cv+1
                continue
            if is_consonant(b):
                cc = cc+1
                continue
                
print("v followed by v: "+str(vv/(vv+vc+cc+cv)))
print("v followed by c: "+str(vc/(vv+vc+cc+cv)))
print("c followed by v: "+str(cv/(vv+vc+cc+cv)))
print("c followed by c: "+str(cc/(vv+vc+cc+cv)))

word count: 370099
avg word length: 9.44254915576643
vowel fraction: 0.3915508095452571
v followed by v: 0.053325584022679534
v followed by c: 0.3586505574030933
c followed by v: 0.35635616830299377
c followed by c: 0.23166769027123335


In [3]:
%autoreload
T = boggle.Tester(all_words, 4)

In [4]:
%autoreload

import datetime
import random as rnd

def printmatrix(ls, size):
    print(ls)
    lls = []
    for r in range(0,size):
        line = []
        for c in range(0,size):
            line.append(ls[r*size+c])
        lls.append(line)
    print("=====")
    for line in lls:
        print(line)

CHARSET = list('abcdefghijklmnopqrstuvwxyz')
VOWELS = list('aeiou')
CONSONANTS = list('bcdfghjklmnpqrstvwxyz')

def generateRandom(size):
    ls = []
    for i in range(0,size*size):
        ls.append(CHARSET[rnd.randint(0,len(CHARSET)-1)])
    return ls

inputs = []
for i in range(0,1000):
    inputs.append(generateRandom(4))
    
res = T.test_multiple(inputs)
print("avg wordcount: "+str(res.avg_word_num))
print("avg score: "+str(res.avg_score))

avg wordcount: 142.12
avg score: 157.421


In [5]:
def generateVowelRatio(size):
    ls = []
    for i in range(0, size*size):
        x = rnd.random()
        if x<=VOWEL_P:
            ls.append(rnd.choice(VOWELS))
        else:
            ls.append(rnd.choice(CONSONANTS))
    return ls

inputs = []
for i in range(0,1000):
    inputs.append(generateVowelRatio(4))
    
res = T.test_multiple(inputs)
print("avg wordcount: "+str(res.avg_word_num))
print("avg score: "+str(res.avg_score))

avg wordcount: 196.419
avg score: 228.751


In [6]:
def weighted_random_choice(choices):
    m = sum(choices.values())
    pick = rnd.uniform(0, m)
    current = 0
    for key, value in choices.items():
        current += value
        if current > pick:
            return key
     
def generateFromDistribution(size):
    ls = []
    for i in range(0,size*size):
        ls.append(weighted_random_choice(LETTER_DISTR))
    return ls
    
inputs = []
for i in range(0,1000):
    inputs.append(generateFromDistribution(4))
       
res = T.test_multiple(inputs)
print("avg wordcount: "+str(res.avg_word_num))
print("avg score: "+str(res.avg_score))

avg wordcount: 404.35
avg score: 576.665


In [7]:
VOWEL_DISTR = {}
CONSONANT_DISTR = {}
for x in LETTER_DISTR:
    if x in VOWELS:
        VOWEL_DISTR[x]=LETTER_DISTR[x]
    else:
        CONSONANT_DISTR[x]=LETTER_DISTR[x]

def get_ratio(ls):
    v_num = 0
    if len(ls) == 0:
        return 0
    for x in ls:
        if x in VOWELS:
            v_num = v_num+1 
    return v_num/len(ls)
        
def generateFromDistributionBalanced(size):
    ls = []
    for i in range(0, size*size):
        v_ratio = get_ratio(ls)
        if v_ratio<VOWEL_P:
            ls.append(weighted_random_choice(VOWEL_DISTR))
        else:
            ls.append(weighted_random_choice(CONSONANT_DISTR))
    rnd.shuffle(ls)
    return ls
           
inputs = []
for i in range(0,1000):
    inputs.append(generateFromDistributionBalanced(4))
       
res = T.test_multiple(inputs)
print("avg wordcount: "+str(res.avg_word_num))
print("avg score: "+str(res.avg_score))


avg wordcount: 443.186
avg score: 652.858


In [8]:
import matplotlib.pyplot as plt

def generateFromVowelNumber(size, vowel_number):
    ls = []
    for i in range(0,vowel_number):
        ls.append(weighted_random_choice(VOWEL_DISTR))
    for i in range(vowel_number, size*size):
        ls.append(weighted_random_choice(CONSONANT_DISTR))
    rnd.shuffle(ls)
    return ls

counts = []
scores = []
for i in range(1,16):
    inputs = []
    for i in range(0,1000):
        inputs.append(generateFromVowelNumber(4,i))
    res = T.test_multiple(inputs)
    counts.append(res.avg_word_num)
    scores.append(res.avg_score)
    
plt.plot(counts)
plt.plot(scores)
print(np.amax(counts))
print(np.amax(scores))

100.66
103.635


In [9]:
def c2i(c):
    return ord(c)-ord('a')

def i2c(i):
    return chr(i+ord('a'))

M2 = np.zeros((26,26))
count = 0
for w in all_words:
    count = count+len(w)-1
    for i in range(0,len(w)-1):
        M2[c2i(w[i]),c2i(w[i+1])] = M2[c2i(w[i]),c2i(w[i+1])]+1
  
for r in range(0,26):
    for c in range(0,r):
        M2[r,c] = M2[r,c]+M2[c,r]
        M2[c,r] = M2[r,c]

In [10]:
import datetime

size = 4
G = boggle.BoggleGraph(size)
    
def weighted_random_choice_arg(distribution):
    m = sum(distribution)
    pick = rnd.uniform(0, m)
    current = 0
    for i in range(0,len(distribution)):
        current += distribution[i]
        if current > pick:
            return i
        
def generateFromAdjacencyDistribution(size, matrix, adjacency):
    out = []
    for i in range(0,size*size):
        out.append('_')
    #fringe node is (index, parent_char)
    fringe = []
    fringe.append( (rnd.randint(0, size*size-1), c2i(weighted_random_choice(LETTER_DISTR))) )
    to_visit = np.arange(0,size*size, 1).tolist()
    while len(fringe) > 0:
        #pick
        i = rnd.randint(0,len(fringe)-1)
        cur = fringe.pop(i)
        if cur[0] not in to_visit:
            continue
        to_visit.remove(cur[0])
        #visit
        l = weighted_random_choice_arg(matrix[cur[1]])
        out[cur[0]] = i2c(l)
        #succ
        succ = list(filter(lambda x: x in to_visit, adjacency[cur[0]]))
        for s in succ:
            fringe.append( (s,l) )
    return out

t1 = datetime.datetime.now()
inputs = []
for i in range(0,1000):
    inputs.append(generateFromAdjacencyDistribution(4,M2,G.adjacency))
t2 = datetime.datetime.now()
    
res = T.test_multiple(inputs)
print("avg wordcount: "+str(res.avg_word_num))
print("avg score: "+str(res.avg_score))
print("avg generation time: "+str((t2-t1)/1000))

avg wordcount: 522.862
avg score: 822.867
avg generation time: 0:00:00.000347


In [24]:
def combineDistributions(distributionList):
    ls = []
    for i in range(0,len(distributionList[0])):
        v = 1
        for j in range(0, len(distributionList)):
            v = v*(distributionList[j])[i]
        ls.append(v)
    return ls
        
def generateFromCombinedAdjacency(size, matrix, adjacency, seed_size):
    out = []
    distributions = []
    for i in range(0,size*size):
        out.append('_')
        distributions.append([])
    fringe = [] #fringe node is (index)
    
    #seeding and seed visit
    seed = []
    indexes = np.arange(0,size*size, 1)
    rnd.shuffle(indexes)
    to_visit = np.arange(0,size*size, 1).tolist()
    for i in range(0, seed_size):    
        cur = indexes[i]
        to_visit.remove(cur)
        l = weighted_random_choice_arg(matrix[cur])
        out[cur] = i2c(l)
        #succ
        succ = list(filter(lambda x: x in to_visit, adjacency[cur]))
        fringe.extend(succ)
        for s in succ:
            distributions[s].append(matrix[l])
    
    #main loop
    while len(fringe) > 0:
        #pick
        cur = fringe.pop(0)
        if cur not in to_visit:
            continue
        to_visit.remove(cur)
        #visit
        distr_list = distributions[cur]
        l = weighted_random_choice_arg(combineDistributions(distr_list))
        out[cur] = i2c(l)
        #succ
        succ = list(filter(lambda x: x in to_visit, adjacency[cur]))
        fringe.extend(succ)
        for s in succ:
            distributions[s].append(matrix[l])
    return out

t1 = datetime.datetime.now()
inputs = []
for i in range(0,1000):
    inputs.append(generateFromCombinedAdjacency(4,M2,G.adjacency, 3))
t2 = datetime.datetime.now()
    
res = T.test_multiple(inputs)
print("avg wordcount: "+str(res.avg_word_num))
print("avg score: "+str(res.avg_score))
print("avg generation time: "+str((t2-t1)/1000))

avg wordcount: 950.502
avg score: 1954.637
avg generation time: 0:00:00.000552


In [30]:
w = []
s = []
for x in range(1,15):
    inputs = []
    for i in range(0,1000):
        inputs.append(generateFromCombinedAdjacency(4,M2,G.adjacency, x))
    res = T.test_multiple(inputs)
    w.append(res.avg_word_num)
    s.append(res.avg_score)

In [35]:

print(len(WD)-len(SC))

0
